<a href="https://colab.research.google.com/github/timomatthies/html-grundstruktur/blob/master/Vorlage_HyperLand_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geemap
import random

ee.Authenticate()
ee.Initialize(project='ee-timomatthies17')

#Insert point and buffer here
point = 1.577, 28.337
buffer = 50000
buffer_test = 50

# Define a region of interest
roi = ee.Geometry.Point(point[1], point[0])  # Create a buffer around the point

point_lat = point[0]
point_long = point[1]

print("Point Latitude:", point_lat)
print("Point Longitude:", point_long)

#Load and filter Dataset, apply scaling factors
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

l8_filtered = l8 \
  .filterDate('2021-05-01', '2021-09-01') \
  .filterBounds(roi)
  #.filterMetadata('CLOUD_COVER', 'less_than', 10) \
  #.sort("CLOUD_COVER") \


# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# Apply scale factors to the dataset
l8_filtered = l8_filtered.map(apply_scale_factors)

# Construct an Image from the Dataset
my_image = l8_filtered.first()

print(type(my_image))
print(type(l8_filtered))

#selected bands
bands = ['SR_B2', 'SR_B6', 'SR_B5']

# Visualization parameters
visualization = {
    'bands': bands,
     #'bands': random_elements,
    'min': 0.0,
    'max': 1,
    'gamma': 1,
}

# my_image_vis = my_image.visualize(**viz_params)

Map = geemap.Map(center=(point[0],point[1]), zoom=12, height=1000)
Map.addLayer(my_image, visualization, 'Timo Color Image')
# Map.addLayer(l8_filtered, viz_params, 'Current Test: ' + str(viz_params['bands'][0]))
# Map.add_basemap("SATELLITE")
# Map.add_basemap("OpenTopoMap")
Map

In [ ]:
# Get image properties
props = geemap.image_props(my_image)

# Get information about the properties
props_info = props.getInfo()

# Access the system:index property
system_id = props_info.get('system:id')

filename = ("HL_" +
            'LAT-' + str(point[0]) + # Latitude
            '_LONG-' + str(point[1]) + # Longitude
            '_BUF-' + str(buffer) + # Buffer
            "_BANDS-" +
            str(bands[0]).replace('_', '') + "/" + str(bands[1]).replace('_', '') + "/" + str(bands[0]).replace('_', '') + # Bands
            "_ID-" +
            system_id) # Image ID

In [ ]:
# Export image to Google Drive
export_params = {
    'image': my_image.visualize(**visualization),  # Fix the visualization part
    'description': 'export',
    'folder': 'EarthEngine_Export',  # Specify your Google Drive folder
    'fileNamePrefix': filename,
    'region': roi.buffer(buffer),
    'scale': 10,  # Set the scale (resolution) of the exported image
}
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

print('Exporting image to Google Drive...')

Exporting image to Google Drive...


In [ ]:
# Initialize counter
export_counter = 1

# Function to export image to Google Drive
def export_image(image):
    global export_counter  # Declare export_counter as a global variable

    # Construct file name with iteration variable
    file_name = 'image-' + str(export_counter)

    # Export the image to Google Drive
    export_params = {
        'image': image,
        'description': file_name,
        'folder': 'GEE_Export',  # Specify the folder in your Google Drive where you want to save the exported image
        'scale': 30,  # Adjust scale as needed
        'region': roi.buffer(buffer_test)
    }

    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    # Increment the export counter
    export_counter += 1

export_image(my_image)